In [1]:
!nvidia-smi

Fri Apr 26 06:50:24 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla P100-PCIE-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              26W / 250W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.9 MB/s eta 0:00:00


In [3]:
from sklearn.metrics import f1_score
import numpy as np
import torch


label_dict= {'neutral': 0, 'surprise': 1, 'fear': 2, 'sadness': 3, 'joy': 4, 'disgust': 5, 'anger': 6}
def f1_score_func(preds,labels):
    preds_flat=np.argmax(preds,axis=1).flatten()
    labels_flat=labels.flatten()
    return f1_score(labels_flat,preds_flat,average="weighted")

def accuracy_per_class(preds,labels):
    label_dict_inverse={v:k for k, v in label_dict.items()}
    preds_flat=np.argmax(preds,axis=1).flatten()
    labels_flat=labels.flatten()
    totalacc=0
    tot=0
    for label in np.unique(labels_flat):
        y_preds=preds_flat[labels_flat==label]
        y_true=labels_flat[labels_flat==label]
        totalacc+=len(y_preds[y_preds == label])
        tot+=len(y_true)
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds == label])}/{len(y_true)}\n')
    print("Acc=",totalacc/tot)

In [4]:
import random
import numpy as np
from tqdm import tqdm
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import torch
import pandas as pd
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import f1_score

# Initialize RoBERTa Tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Initialize RoBERTa Model for Sequence Classification
model = RobertaForSequenceClassification.from_pretrained("roberta-base")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
import random
import numpy as np
import torch

seed_val = 994
device = torch.device("cuda")
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

batch_size = 16
epochs = 5


In [7]:
import pandas as pd
from transformers import RobertaTokenizer

# URLs to the raw CSV files on GitHub
train_url = "https://raw.githubusercontent.com/declare-lab/MELD/master/data/MELD/train_sent_emo.csv"
dev_url = "https://raw.githubusercontent.com/declare-lab/MELD/master/data/MELD/dev_sent_emo.csv"
test_url = "https://raw.githubusercontent.com/declare-lab/MELD/master/data/MELD/test_sent_emo.csv"

# Load data from the raw CSV URLs
train_data = pd.read_csv(train_url)
dev_data = pd.read_csv(dev_url)
test_data = pd.read_csv(test_url)

# Extract utterances
utterances_train = train_data["Utterance"]
utterances_dev = dev_data["Utterance"]
utterances_test = test_data["Utterance"]

# Initialize RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Mapping labels to integers
possible_labels = train_data["Emotion"].unique()
label_dict = {label: idx for idx, label in enumerate(possible_labels)}

# Replace labels with their corresponding integers
train_data["Emotion"] = train_data["Emotion"].replace(label_dict)
dev_data["Emotion"] = dev_data["Emotion"].replace(label_dict)
test_data["Emotion"] = test_data["Emotion"].replace(label_dict)

# Processed data for RoBERTa tokenizer
processed_data_train = [tokenizer.encode(text, add_special_tokens=True) for text in utterances_train]
processed_data_dev = [tokenizer.encode(text, add_special_tokens=True) for text in utterances_dev]
processed_data_test = [tokenizer.encode(text, add_special_tokens=True) for text in utterances_test]


/tmp/ipykernel_34/2155645751.py:27: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_data["Emotion"] = train_data["Emotion"].replace(label_dict)
/tmp/ipykernel_34/2155645751.py:28: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dev_data["Emotion"] = dev_data["Emotion"].replace(label_dict)
/tmp/ipykernel_34/2155645751.py:29: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future 

In [9]:
# Tokenizing data
encoded_inputs_train = tokenizer.batch_encode_plus(
    [str(text) for text in utterances_train],  # Ensure each text is converted to string
    add_special_tokens=True,
    return_attention_mask=True,
    padding=True,
    max_length=50,
    truncation=True,
    return_tensors='pt'
)

encoded_inputs_dev = tokenizer.batch_encode_plus(
    [str(text) for text in utterances_dev],  # Ensure each text is converted to string
    add_special_tokens=True,
    return_attention_mask=True,
    padding=True,
    max_length=50,
    truncation=True,
    return_tensors='pt'
)

encoded_inputs_test = tokenizer.batch_encode_plus(
    [str(text) for text in utterances_test],  # Ensure each text is converted to string
    add_special_tokens=True,
    return_attention_mask=True,
    padding=True,
    max_length=50,
    truncation=True,
    return_tensors='pt'
)

# Extract input IDs, attention masks, and labels
input_ids_train = encoded_inputs_train["input_ids"]
attention_mask_train = encoded_inputs_train["attention_mask"]
labels_train = torch.tensor(train_data.Emotion.values)

input_ids_dev = encoded_inputs_dev["input_ids"]
attention_mask_dev = encoded_inputs_dev["attention_mask"]
labels_dev = torch.tensor(dev_data.Emotion.values)

input_ids_test = encoded_inputs_test["input_ids"]
attention_mask_test = encoded_inputs_test["attention_mask"]
labels_test = torch.tensor(test_data.Emotion.values)


In [10]:
from transformers import RobertaForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Define the RoBERTa model
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=7)

# Move the model to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define the training, validation, and test datasets
dataset_train = TensorDataset(input_ids_train, attention_mask_train, labels_train)
dataset_dev = TensorDataset(input_ids_dev, attention_mask_dev, labels_dev)
dataset_test = TensorDataset(input_ids_test, attention_mask_test, labels_test)

# Define the dataloaders
kwargs = {'num_workers': 1, 'pin_memory': True} if device == "cuda" else {}
dataloader_train = DataLoader(dataset_train, sampler=RandomSampler(dataset_train), batch_size=batch_size, **kwargs)
dataloader_dev = DataLoader(dataset_dev, sampler=SequentialSampler(dataset_dev), batch_size=batch_size, **kwargs)
dataloader_test = DataLoader(dataset_test, sampler=SequentialSampler(dataset_test), batch_size=len(test_data), **kwargs)

# Define optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=5e-5, eps=1e-8)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=int(0.1 * len(dataloader_train) * epochs), num_training_steps=len(dataloader_train) * epochs)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [11]:
def evaluate(dataloader_val):
    model.eval()
    loss_val_total = 0
    predictions, true_vals = [], []

    for batch in dataloader_val:
        batch = tuple(b.to(device) for b in batch)

        inputs = {
            "input_ids": batch[0],
            "attention_mask": batch[1],
            "labels": batch[2]
        }

        with torch.no_grad():
            outputs = model(**inputs)

        loss = outputs.loss
        logits = outputs.logits
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs["labels"].cpu().numpy()

        predictions.append(logits)
        true_vals.append(label_ids)

    loss_val_avg = loss_val_total / len(dataloader_val)
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)

    return loss_val_avg, predictions, true_vals


In [12]:
import torch
from tqdm import tqdm
from sklearn.metrics import f1_score
import numpy as np

# Define a function to calculate the F1 score
def f1_score_func(predictions, true_vals):
    preds_flat = np.argmax(predictions, axis=1).flatten()
    labels_flat = true_vals.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

# Define the path to save the models in the Kaggle working directory
model_save_path = "/kaggle/working/"

# Training loop
for epoch in tqdm(range(1, epochs + 1)):
    model.train()
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc=f"Epoch {epoch}", leave=False, disable=False)
    for batch in progress_bar:
        model.zero_grad()

        batch = tuple(b.to(device) for b in batch)

        inputs = {"input_ids": batch[0],
                  "attention_mask": batch[1],
                  "labels": batch[2]}
        outputs = model(**inputs)
        loss = outputs.loss

        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        progress_bar.set_postfix({"training loss": "{:.2f}".format(loss.item() / len(batch))})

    # Save the model's state dictionary to the specified path in Kaggle working directory
    model_save_name = f"finetuned_epoch{epoch}.model"
    torch.save(model.state_dict(), model_save_path + model_save_name)
    tqdm.write(f"Epoch {epoch}")
    loss_train_avg = loss_train_total / len(dataloader_train)
    tqdm.write(f"Training loss: {loss_train_avg}")

    # Evaluation
    val_loss, predictions, true_vals = evaluate(dataloader_dev)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


  0%|          | 0/5 [01:09<?, ?it/s]                                         

Epoch 1
Training loss: 1.294658275127411


 20%|██        | 1/5 [01:11<04:44, 71.16s/it]

Validation loss: 1.3024521367890494
F1 Score (Weighted): 0.5231124352126127



 20%|██        | 1/5 [02:19<04:44, 71.16s/it]                                 

...

Validation loss: 1.2933330753019878
F1 Score (Weighted): 0.5925660741647159



 80%|████████  | 4/5 [05:51<01:10, 70.53s/it]                                 

Epoch 5
Training loss: 0.44081782953143117


100%|██████████| 5/5 [05:53<00:00, 70.65s/it]

Validation loss: 1.4446274042129517
F1 Score (Weighted): 0.5799667134249162


In [15]:
import numpy as np
import torch

def accuracy_per_class(preds, labels, label_dict):
    # Move tensors from GPU to CPU and convert to NumPy arrays
    preds = preds.cpu().numpy()
    labels = labels.cpu().numpy()
    
    # Create inverse label dictionary
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    # Compute flat predictions and labels
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    # Calculate accuracy per class
    total_acc = 0
    for label in np.unique(labels_flat):
        idx = labels_flat == label
        correct_preds = (preds_flat[idx] == labels_flat[idx]).sum()
        total_preds = idx.sum()
        accuracy = correct_preds / total_preds
        print(f'Accuracy for class {label_dict_inverse[label]}: {accuracy}')
        total_acc += accuracy
    
    # Compute average accuracy across all classes
    avg_accuracy = total_acc / len(np.unique(labels_flat))
    print(f'Average accuracy: {avg_accuracy}')

# Example usage:
# Assuming `predictions` and `true_vals` are PyTorch tensors on the GPU (`cuda`)
accuracy_per_class(predictions, true_vals, label_dict)


Accuracy for class neutral: 0.7730891719745223
Accuracy for class surprise: 0.608540925266904
Accuracy for class fear: 0.28
Accuracy for class sadness: 0.3076923076923077
Accuracy for class joy: 0.5970149253731343
Accuracy for class disgust: 0.2647058823529412
Accuracy for class anger: 0.4144927536231884
Average accuracy: 0.4636479951832854


In [16]:
print(len(encoded_inputs_train["attention_mask"]))

9989


In [17]:
import torch
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Function to evaluate model performance
def evaluate_model(model, dataloader):
    # Set model to evaluation mode
    model.eval()

    # Initialize lists to store true labels and predicted labels
    true_labels = []
    predicted_labels = []

    # Iterate over batches
    for batch in dataloader:
        # Extract inputs and labels from batch
        input_ids, attention_mask, labels = batch

        # Move inputs and labels to device
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        # Forward pass
        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

        # Compute predicted labels
        _, predicted = torch.max(logits, dim=1)

        # Append true labels and predicted labels
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())

    # Compute accuracy
    accuracy = accuracy_score(true_labels, predicted_labels)

    # Generate classification report
    class_report = classification_report(true_labels, predicted_labels)

    # Generate confusion matrix
    conf_matrix = confusion_matrix(true_labels, predicted_labels)

    return accuracy, class_report, conf_matrix

# Evaluate the model on the test set
accuracy, class_report, conf_matrix = evaluate_model(model, dataloader_test)

# Print the performance metrics
print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(class_report)
print("Confusion Matrix:")
print(conf_matrix)


Accuracy: 0.621072796934866
Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.77      0.76      1256
           1       0.51      0.61      0.56       281
           2       0.31      0.28      0.29        50
           3       0.40      0.31      0.35       208
           4       0.55      0.60      0.57       402
           5       0.34      0.26      0.30        68
           6       0.49      0.41      0.45       345

    accuracy                           0.62      2610
   macro avg       0.48      0.46      0.47      2610
weighted avg       0.61      0.62      0.62      2610

Confusion Matrix:
[[971  54  15  55  93  12  56]
 [ 44 171   1   4  33   4  24]
 [ 18   5  14   6   2   0   5]
 [ 81   8   7  64  20   5  23]
 [ 83  35   3   9 240   3  29]
 [ 26   6   0   1   5  18  12]
 [ 69  56   5  20  41  11 143]]


In [18]:
import torch
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Function to evaluate model performance
def evaluate_model(model, dataloader):
    # Set model to evaluation mode
    model.eval()

    # Initialize lists to store true labels and predicted labels
    true_labels = []
    predicted_labels = []

    # Define emotion labels
    emotion_labels = ['neutral', 'surprise', 'fear', 'sadness', 'joy', 'disgust', 'anger']

    # Iterate over batches
    for batch in dataloader:
        # Extract inputs and labels from batch
        input_ids, attention_mask, labels = batch

        # Move inputs and labels to device
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        # Forward pass
        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

        # Compute predicted labels
        _, predicted = torch.max(logits, dim=1)

        # Append true labels and predicted labels
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())

    # Compute accuracy
    accuracy = accuracy_score(true_labels, predicted_labels)

    # Generate classification report with emotion labels
    class_report = classification_report(true_labels, predicted_labels, target_names=emotion_labels)

    # Generate confusion matrix
    conf_matrix = confusion_matrix(true_labels, predicted_labels)

    return accuracy, class_report, conf_matrix

# Evaluate the model on the test set
accuracy, class_report, conf_matrix = evaluate_model(model, dataloader_test)

# Print the performance metrics
print(f"Accuracy of roberta-base model: {accuracy}")
print("Classification Report:")
print(class_report)
print("Confusion Matrix:")
print(conf_matrix)


Accuracy of roberta-base model: 0.621072796934866
Classification Report:
              precision    recall  f1-score   support

     neutral       0.75      0.77      0.76      1256
    surprise       0.51      0.61      0.56       281
        fear       0.31      0.28      0.29        50
     sadness       0.40      0.31      0.35       208
         joy       0.55      0.60      0.57       402
     disgust       0.34      0.26      0.30        68
       anger       0.49      0.41      0.45       345

    accuracy                           0.62      2610
   macro avg       0.48      0.46      0.47      2610
weighted avg       0.61      0.62      0.62      2610

Confusion Matrix:
[[971  54  15  55  93  12  56]
 [ 44 171   1   4  33   4  24]
 [ 18   5  14   6   2   0   5]
 [ 81   8   7  64  20   5  23]
 [ 83  35   3   9 240   3  29]
 [ 26   6   0   1   5  18  12]
 [ 69  56   5  20  41  11 143]]


In [20]:
# User input sentence
user_sentence = "I hope that I become intelligent enough to do this project on my own."

# Tokenize the input sentence
encoded_user_input = tokenizer.encode_plus(
    user_sentence,
    add_special_tokens=True,
    max_length=50,
    truncation=True,
    padding='max_length',
    return_tensors='pt'
)

# Move input tensors to the appropriate device (e.g., CUDA if available)
input_ids = encoded_user_input['input_ids'].to(device)
attention_mask = encoded_user_input['attention_mask'].to(device)

# Pass the input tensors through the model to obtain predictions
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)

# Extract predicted class probabilities
probabilities = torch.softmax(outputs.logits, dim=1).squeeze()

# Determine the predicted class
predicted_class = torch.argmax(probabilities).item()

# Map the predicted class index to the corresponding emotion label
predicted_emotion = list(label_dict.keys())[list(label_dict.values()).index(predicted_class)]

# Print the predicted emotion
print("Predicted emotion:", predicted_emotion)


Predicted emotion: sadness


In [21]:
from sklearn.metrics import classification_report, confusion_matrix

# List of sentences
sentences = [
    "I feel betrayed, I am gonna do this project alone.",
    "This is really weird person.",
    "Damn! Them weirdos always creating fuss!",
    "I am overjoyed to tell you that I got a job.",
    "Hi! You look so handsome today!",
    "I feel so great after exams."
]

# Initialize lists to store predictions and scores
predicted_emotions = []
predicted_scores = []

# Process each sentence
for sentence in sentences:
    # Tokenize the input sentence
    encoded_input = tokenizer.encode_plus(
        sentence,
        add_special_tokens=True,
        max_length=50,
        truncation=True,
        padding='max_length',
        return_tensors='pt'
    )

    # Move input tensors to the appropriate device (e.g., CUDA if available)
    input_ids = encoded_input['input_ids'].to(device)
    attention_mask = encoded_input['attention_mask'].to(device)

    # Pass the input tensors through the model to obtain predictions
    with torch.no_grad():
        outputs = model(input_ids, attention_mask)

    # Extract predicted class probabilities
    probabilities = torch.softmax(outputs.logits, dim=1).squeeze()

    # Determine the predicted class index and corresponding emotion label
    predicted_class = torch.argmax(probabilities).item()
    predicted_emotion = list(label_dict.keys())[list(label_dict.values()).index(predicted_class)]

    # Store predicted emotion and score
    predicted_emotions.append(predicted_emotion)
    predicted_scores.append(probabilities[predicted_class].item())

# Print predicted emotions and scores for each sentence
for i, (sentence, emotion, score) in enumerate(zip(sentences, predicted_emotions, predicted_scores), 1):
    print(f"Sentence {i}: {sentence}")
    print(f"Predicted Emotion: {emotion} (Score: {score:.4f})")
    print()


Sentence 1: I feel betrayed, I am gonna do this project alone.
Predicted Emotion: sadness (Score: 0.9791)

Sentence 2: This is really weird person.
Predicted Emotion: disgust (Score: 0.8970)

Sentence 3: Damn! Them weirdos always creating fuss!
Predicted Emotion: anger (Score: 0.9393)

Sentence 4: I am overjoyed to tell you that I got a job.
Predicted Emotion: joy (Score: 0.9841)

Sentence 5: Hi! You look so handsome today!
Predicted Emotion: joy (Score: 0.9891)

Sentence 6: I feel so great after exams.
Predicted Emotion: joy (Score: 0.9748)



**Distillbert For Meld Dataset**

In [22]:
!pip install transformers
import torch
import numpy as np
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler
from sklearn.metrics import f1_score, accuracy_score


In [31]:
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader
from transformers import DistilBertTokenizer
from sklearn.preprocessing import LabelEncoder

# URLs to the raw CSV files on GitHub
train_url = "https://raw.githubusercontent.com/declare-lab/MELD/master/data/MELD/train_sent_emo.csv"
dev_url = "https://raw.githubusercontent.com/declare-lab/MELD/master/data/MELD/dev_sent_emo.csv"
test_url = "https://raw.githubusercontent.com/declare-lab/MELD/master/data/MELD/test_sent_emo.csv"

# Load data from the raw CSV URLs
train_data = pd.read_csv(train_url)
dev_data = pd.read_csv(dev_url)
test_data = pd.read_csv(test_url)

# Extract utterances and labels
utterances_train = train_data["Utterance"]
utterances_dev = dev_data["Utterance"]
utterances_test = test_data["Utterance"]

# Initialize DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenize and encode the data for training set
encoded_inputs_train = tokenizer(list(utterances_train), padding=True, truncation=True, max_length=128, return_tensors='pt')
input_ids_train = encoded_inputs_train['input_ids']
attention_masks_train = encoded_inputs_train['attention_mask']

# Convert emotion labels to numeric categories
label_encoder = LabelEncoder()
labels_train = torch.tensor(label_encoder.fit_transform(train_data["Emotion"].values))

# Tokenize and encode the data for development set
encoded_inputs_dev = tokenizer(list(utterances_dev), padding=True, truncation=True, max_length=128, return_tensors='pt')
input_ids_dev = encoded_inputs_dev['input_ids']
attention_masks_dev = encoded_inputs_dev['attention_mask']
labels_dev = torch.tensor(label_encoder.transform(dev_data["Emotion"].values))

# Tokenize and encode the data for test set
encoded_inputs_test = tokenizer(list(utterances_test), padding=True, truncation=True, max_length=128, return_tensors='pt')
input_ids_test = encoded_inputs_test['input_ids']
attention_masks_test = encoded_inputs_test['attention_mask']
labels_test = torch.tensor(label_encoder.transform(test_data["Emotion"].values))

# Create datasets for training, development, and test sets
train_dataset = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dev_dataset = TensorDataset(input_ids_dev, attention_masks_dev, labels_dev)
test_dataset = TensorDataset(input_ids_test, attention_masks_test, labels_test)

# Define batch size
batch_size = 32

# Create data loaders for training, development, and test sets
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


In [32]:
# Model configuration
num_labels = len(train_data["Emotion"].unique())  # Number of unique emotion labels

# Initialize DistilBERT model for sequence classification
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=num_labels)

# Set device (GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define training parameters
batch_size = 16
epochs = 5
optimizer = AdamW(model.parameters(), lr=2e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(input_ids_train) * epochs)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [33]:
# Create TensorDatasets
train_dataset = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dev_dataset = TensorDataset(input_ids_dev, attention_masks_dev, labels_dev)
test_dataset = TensorDataset(input_ids_test, attention_masks_test, labels_test)

# Create DataLoader
train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=batch_size)
dev_dataloader = DataLoader(dev_dataset, sampler=SequentialSampler(dev_dataset), batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size=batch_size)


In [39]:
import torch
from tqdm import tqdm
from sklearn.metrics import f1_score
import numpy as np

# Define a function to calculate the F1 score
def f1_score_func(predictions, true_vals):
    preds_flat = np.argmax(predictions, axis=1).flatten()
    labels_flat = true_vals.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

# Define the path to save the models in Kaggle's working directory
model_save_path = "/kaggle/working/"

# Define the evaluate function
def evaluate(model, dataloader):
    model.eval()
    loss_val_total = 0
    predictions, true_vals = [], []

    for batch in dataloader:
        batch = tuple(b.to(device) for b in batch)

        with torch.no_grad():
            inputs = {"input_ids": batch[0],
                      "attention_mask": batch[1],
                      "labels": batch[2]}
            outputs = model(**inputs)
            loss = outputs.loss
            logits = outputs.logits

            loss_val_total += loss.item()
            predictions.append(logits.detach().cpu().numpy())
            true_vals.append(inputs['labels'].detach().cpu().numpy())

    loss_val_avg = loss_val_total / len(dataloader)
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)

    return loss_val_avg, predictions, true_vals

# Training loop
for epoch in tqdm(range(1, epochs + 1)):
    model.train()
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc=f"Epoch {epoch}", leave=False, disable=False)
    for batch in progress_bar:
        model.zero_grad()

        batch = tuple(b.to(device) for b in batch)

        inputs = {"input_ids": batch[0],
                  "attention_mask": batch[1],
                  "labels": batch[2]}
        outputs = model(**inputs)
        loss = outputs.loss

        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        progress_bar.set_postfix({"training loss": "{:.2f}".format(loss.item() / len(batch))})

    # Save the model's state dictionary to the specified path
    model_save_name = f"distilbert_finetuned_epoch{epoch}.model"
    torch.save(model.state_dict(), model_save_path + model_save_name)
    tqdm.write(f"Epoch {epoch}")
    loss_train_avg = loss_train_total / len(dataloader_train)
    tqdm.write(f"Training loss: {loss_train_avg}")

    # Evaluation
    val_loss, predictions, true_vals = evaluate(model, dataloader_dev)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


  0%|          | 0/5 [00:57<?, ?it/s]                                         

Epoch 1
Training loss: 0.3727062722623348


 20%|██        | 1/5 [00:59<03:56, 59.19s/it]

Validation loss: 1.4446274042129517
F1 Score (Weighted): 0.5799667134249162



 20%|██        | 1/5 [01:56<03:56, 59.19s/it]                                 

...

Validation loss: 1.4446274042129517
F1 Score (Weighted): 0.5799667134249162



 80%|████████  | 4/5 [04:52<00:58, 58.76s/it]                                 

Epoch 5
Training loss: 0.36778048370480537


100%|██████████| 5/5 [04:54<00:00, 58.80s/it]

Validation loss: 1.4446274042129517
F1 Score (Weighted): 0.5799667134249162


In [54]:
import torch
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Function to evaluate model performance
def evaluate_model(model, dataloader):
    # Set model to evaluation mode
    model.eval()

    # Initialize lists to store true labels and predicted labels
    true_labels = []
    predicted_labels = []

    # Iterate over batches
    for batch in dataloader:
        # Extract inputs and labels from batch
        input_ids, attention_mask, labels = batch

        # Move inputs and labels to device
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        # Forward pass
        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

        # Compute predicted labels
        _, predicted = torch.max(logits, dim=1)

        # Append true labels and predicted labels
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())

    # Compute accuracy
    accuracy = accuracy_score(true_labels, predicted_labels)

    # Generate classification report
    class_report = classification_report(true_labels, predicted_labels)

    # Generate confusion matrix
    conf_matrix = confusion_matrix(true_labels, predicted_labels)

    return accuracy, class_report, conf_matrix

# Evaluate the model on the test set
accuracy, class_report, conf_matrix = evaluate_model(model, dataloader_test)

# Print the performance metrics
print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(class_report)
print("Confusion Matrix:")
print(conf_matrix)


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
